In [13]:
#|echo: false
#|output: false
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install manim

Defaulting to user installation because normal site-packages is not writeable


Manim is a *programatic* animation software which separates it from other animation software you might be familiar with. A common way to make animations is using PowerPoint or LibreOffice Present where text or images are placed on the slide and you can select options for how they should move, fade in, rotate, and transform. This kind of *graphical* animation software shows you the objects you are working with, how they are aranged. It allows you to move them around with your mouse and select animations from a toolbar or contextual menu. Manim, by contrast, is *programatic* in the sense that animations are created by writing source code which you render later. Instead of creating a circle by clicking a tool and drawing it to be the size you want, in Manim you define a circle with particular parameters like `Circle(radius=2)` which creates a circle with a radius of 2 units. To animate the circle, you transform that circle *object* using functions. At the end, you render the circle and *then* you get to see how the animation looks.

This probably sounds more complicated than just making some slides and animating the text and drawings there, and you'd be right! Manim gives you fine-grained control over your drawings and animations, but this creative freedom translates to a complex and powerful tool. For simple animations, other software will probably be sufficient, and you should consider the trade-offs when choosing an animation software. All that said, Manim allows you to create engaging and professional-looking visualizations and with some time and dedication, you will be able to make compelling animations in an afternoon!

In this chapter, we will explore the basic building blocks of Manim and develop an intuition for how to programatically manipulate objects. 

## Object-oriented programming
If you've done some programming you might have heard of **object-oriented programming** (OOP). A full description is beyond the scope of this tutorial, but what is important to understand is that OOP is a paradigm which focuses on *object* and how to manipulate them, rather than the functions and logic underlying the object. To illustrate the difference between OOP and functional programming, let's consider an example.

You have a big enrollment and want some way to keep track of your students' information. For some reason you decide to use python for this task. Your roster looks like the following:
* Amy Abstract, 2028, Astronomy
* Billy Bar, 2027, Botany
* Catherine Class, 2028, Chemistry
* David Demo, 2024, Divinity Studies
* Emelia Example, 2025, Economics

How would you store this information in python? Put another way, what would your data structure be? A reasonable solution would be a dictionary. The keys are student names and the values are an ordered pair (important that his be a list because tuples are immutable) with the year first and major second.


In [ ]:
roster = {
    'Amy Abstract': [2028,'Astronomy'],
    'Billy Bar': [2027, 'Botany'],
    'Catherine Class': [2028, 'Chemistry'],
    'David Demo': [2024, 'Divinity Studies'],
    'Emilia Example': [2025, 'Economics']
}

And this works well for a while. Any time you need to look up information on a student, you just run `roster[name]` to get their major and year. But as the term goes on, you realize that your information needs to be updated. David is graduating early. Billy changed his major to Biology. A student comes out to you as non-binary and wants to be referred to as A Abstract instead of their former name (you should probably also add a field for pronouns too). You don't want to have to go in and edit the whole dictionary by hand, so how do you update the dictioanry programmatically? One common solution might look like the following code.

In [ ]:
roster['David Demo'][0] = 2022
roster['Billy Bar'][1] = 'Biology'
roster['A Abstract'] = roster.pop('Amy Abstract')

And this sorta works, but it's not much better than editing the dictionary by hand. How would you even add a field for pronouns to this? You'd have to extend the tuple for every entry, and at that point you might as well just rewrite the dictionary definition from scratch. The code is also not very clear and might be confusing later on, for example we use `[0]` and `[1]` to identify year and major which isn't obvious. There's got to be a better way!

The solution is to reconsider our data structure and how we decided to store this information in the first place. What if, instead of representing each student as a list of entries, what if we treated them as an object: a bundle of data and functions which operate on that data. In Python, objects usually take the form of a **class**, and below we have an example class, `Student`, which we can use to represent students enrolled in our class.

In [ ]:
#|code-line-numbers: true
class Student(object):  
    def __init__(self, name, year, major):
        """When a new student is created, 
        we store the information given on creation
        """
        self.name  = name
        self.year  = year
        self.major = major
        
    def change_major(self, new_major):
        self.major = new_major
    
    def advance_year(self):
        self.year += 1
        
    def change_name(self, new_name):
        self.name = new_name

This is more complicated than the dictionary-of-lists approach, but it is also far more powerful and intuitive. We can see that once we start to use it. Let's initialize our data.

In [ ]:
#|code-line-numbers: true
a = Student('Amy Abstract', 2028, 'Astronomy')
b = Student('Billy Bar', 2027, 'Botany')
c = Student('Catherine Class', 2028, 'Chemistry')
d = Student('David Demo', 2024, 'Divinity Studies')
e = Student('Emilia Example', 2025, 'Economics')
roster = [a,b,c,d,e]

While the set-up isn't much better than the dictionary solution, it does have the benefit of being clear about *what* these objects are: students. As a dictionary, that was information you needed to already know if you wanted to understand the data structure, but here, each packet of data is explicitly identified as a student as a side-effect of instnatiating the class. The benefits of this approach become more obvious once we start making changes later in the term.

In [ ]:
#|code-line-numbers: true
d.year = 2022
b.change_major('Biology')
a.change_name('A Abstract')
a.pronouns = 'they/them'

In a lot of ways, this is similar to our example using a dictionary, but notice how clear this code is. It's obvious that we're changing David's graduation year because we're changing the value of `year`. It's obvious we're changing the major of Billy and the name of A because we are using functions that *tell us* what they are doing. You'll also notice that even though we didn't define pronouns in the class, we can simply add A's pronouns after the fact without affecting the data structure for the rest of the students. That said, adding attributes after-the-fact like this is a bad idea. It's better to modify the class to include a pronouns attribute with a default value and update it later if needed.

### Major principles in OOP (Patterns)
The example given above illustrates a number of principles which will be helpful to keep in mind when working with Manim, structuring your code, and manipulating objects. In software programming, similar problems arise all the time, and certain solutions have shown to be better than others. These solutions are called **patterns** and they articulate principles of good software design. Patterns contrast with **anti-patterns** which are attractive solutions (like our dictionary roster) which ultimately cause more problems later on. In this section we explore some OOP principles and design patterns.

* **Abstraction** focuses on abstracting the *implementation* of a function from the *interface* of the function. For example, a `Student.change_name()` is a function which changes the `Student.name` attribute. How it does this is unimportant, and in fact, we can change it at any time. By creating a function instead of changing the attribute directly, we have created a level of abstraction. We can now call `Student.change_name()` and *always* know that no matter what, the student's name will be changed---we can change names without needing to worry about *how* to change names. 
* **Encapsulation** is a design pattern which focuses on keeping objects self-contained. Each object should have everything it needs to work inside of it already, and things outside the object (users, programs, other objects) should only ever interact with the object through well defined and predictable methods. Changing the name of one student-object should not affect the others, for example. By encapsulating objects, we prevent unintended or unknown data corruption and make debugging easier by limiting how parts of a program are allowed to interact.
* **Inheritance** represents hierarchical relationships between objects. For example, all students are people, but not all people are students. We could have `class Person(object)` which has attributes and functions for names and pronouns, but not major or graduation year. Instead of rewriting all that code for students, we could *inherit* the features of the `Person` class and then *extend* student-specific features in this new class. This is used frequently in Manim and will turn out to be quite powerful. 

## The building blocks of Manim


In [11]:
#|code-line-numbers: true
from manim import *

class SquareExample(Scene):
    def construct(self):
        square = Square()
        self.play(Create(square))
        self.wait(2)

In [12]:
#|echo: false
%manim -v WARNING --disable_caching -qm SquareExample

config.media_width = "75%"
config.media_embed = True